# DATA PREPARATION

We have used four datasets for our analysis and prediction. Each dataset is prepared individually, and joined on a common index of lat, lon, year for each dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import descartes
from itertools import chain
%matplotlib inline

In [2]:
import chart_studio.plotly as py 
from chart_studio.plotly import plot, iplot

import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [1]:
import glob
import json
import os
import re

## COMMON DATA PROCESSING FUNCTIONS

We will first define a common loading procedure that works for most of the datasets considered here.

Let's read in the json file with lat lon and state county mapping for lookup

In [9]:
# Load the Lat/Lon data
with open('data/latlon.json', 'r') as fp:
    lat_lon_json = json.load(fp)

### 1. FUNCTION TO LOAD AND MERGE RAW FILES

In [3]:
def _load(subpath):
    path = os.path.join('data/raw/', subpath, '*.csv')
    all_files = glob.glob(path)

    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None)
        li.append(df)

    return pd.concat(li, axis=0, ignore_index=True)

### 2. FUNCTION TO GET LAT LON PAIRS FROM JSON FILE

In [5]:
def _get_lat_lon(q):
    if q in lat_lon_json:
        return lat_lon_json[q]
    else:
        return [None, None]

### 3. FUNCTION TO CHANGE PRECISION OF LAT LON PAIRS

In [6]:
def _change_precision(a, prec=0):
    acc = []
    for e in a:
        if e is None:
            acc.append(None)
        else:
            acc.append(round(float(e), prec))
    return acc

## DATA READING AND CLEANING FUNCTIONS 

### 1. LOAD TRI DATA

In order to get the TRI dataset into a common format, we will be performing following transformations:

1. Clean the column names to remove spaces and capitals
2. Select relevant columns 
3. Pivot on industry since it is categorical
4. Fix the schema for the columns which will become the index (lat, lon, year)
5. Change precision of lat and lon
6. Regroup by year, lat, lon and aggregate by sum

It's good that the TRI dataset comes with lat lon pair.

In [8]:
# Load file
combined = _load('tri')

# Clean column names
cols = combined.columns
cols = [c.split(' - ')[-1] for c in cols]
cols = [c.split('. ')[-1].lower() for c in cols]
combined.columns = cols
combined = combined.rename(columns={'production wste (8.1-8.7)': 'production waste',
                                        'latitude': 'lat', 'longitude': 'lon'})

# Select Relevant
sel = ['year', 'lat', 'lon', 'industry sector',
           'fugitive air', 'stack air', 'water', 'underground',
           'underground cl i', 'underground c ii-v', 'landfills',
           'rcra c landfill', 'other landfills', 'land treatment',
           'surface impndmnt', 'rcra surface im', 'other surface i',
           'on-site release total', 'trns rlse', 'trns trt',
           'total transfers', 'm10', 'm41', 'm62', 'm40 metal', 'm61 metal', 'm71',
           'm81', 'm82', 'm72', 'm63', 'm66', 'm67', 'm64', 'm65', 'm73', 'm79',
           'm90', 'm94', 'm99', 'off-site release total', 'm20', 'm24', 'm26',
           'm28', 'm93', 'off-site recycled total', 'm56', 'm92',
           'm40 non-metal', 'm50', 'm54',
           'm61 non-metal', 'm69', 'm95', 'off-site treated total',
           'total transfer', 'total releases', 'releases', 'on-site contained',
           'off-site contain', 'production waste']

combined = combined[sel]
combined.columns = [c.replace(' ', '_') for c in combined.columns]

# Pivot industry
d = pd.get_dummies(combined['industry_sector'], prefix='sector')
df = pd.concat([combined, d], axis=1).drop(['industry_sector'], axis=1)

# Fix schema
df = df.dropna()
df.year = df.year.astype(int)
df.lat = df.lat.astype(int)
df.lon = df.lon.astype(int)

# Shift Precision
df.lat = _change_precision(df.lat)
df.lon = _change_precision(df.lon)

# Group up
df = df.dropna().groupby(by=['year', 'lat', 'lon'], as_index=False).sum()

tri = df.dropna()

### 2. LOAD AQI DATA

In order to get the AQI dataset into a common format, we will be performing following transformations:

1. Clean the column names to remove spaces and capitals
2. Select relevant columns (as defined in function)
3. Convert date to year by remove the month and day
4. Pivot on defining_parameter since it is categorical.
5. Regroup by 'year', 'state_name', 'county_name', aggregating by mean.
6. Convert 'state_name', 'county_name' to lat and lon using the JSON file.
7. Fix the schema for the columns which will become the index (lat, lon, year)
8. Change precision of lat and lon
9. Regroup by year, lat, lon and aggregate by sum

In [ ]:
# Remove NA columns
combined = _load('aqi').dropna(axis=1)

# Select relevant columns
sel = ['State Name', 'county Name', 'Date', 'AQI', 'Defining Parameter']
combined = combined[sel]

# Clean column names
combined.columns = [c.replace(" ", "_").lower() for c in combined.columns]

# Convert date column to year only
combined.date = combined.date.str.split('-').str[0]
combined = combined.rename(columns={'date': 'year'})
combined = combined.astype({'year': 'int64'})

# Pivot defining_parameter column
d = pd.get_dummies(combined['defining_parameter'], prefix='defining')
df = pd.concat([combined, d], axis=1).drop(['defining_parameter'], axis=1)
df = df.dropna().groupby(by=['year', 'state_name', 'county_name'], as_index=False).mean()

# Convert to Lat/Lon
loc = list(df.state_name + ", " + df.county_name)

lat = [_get_lat_lon(i)[0] for i in loc]
lon = [_get_lat_lon(i)[1] for i in loc]

df['lat'] = _change_precision(lat)
df['lon'] = _change_precision(lon)

# Fix Schema
df = df.dropna()
df.year = df.year.astype(int)
df.lat = df.lat.astype(int)
df.lon = df.lon.astype(int)

# Group up
df = df.dropna().groupby(by=['year', 'lat', 'lon'], as_index=False).sum()

aqi=df.dropna()

### 3. LOAD CANCER DATA

In order to get the Cancer dataset into a common format, we will be performing following transformations:

1. Clean the column names to remove spaces and capitals
2. Select relevant columns (as defined in function)
3. Pivot on leading_cancer_sites since it is categorical.
4. Fix the MSA column 
5. Convert the cleaned MSA column to lat and lon.
6. Fix the schema for the columns which will become the index (lat, lon, year)
7. Change precision of lat and lon
8. Regroup by year, lat, lon and aggregate by sum

In [ ]:
path = os.path.join('data/raw/', 'health/United States and Puerto Rico Cancer Statistics, 1999-2016 Incidence.txt')
cancer = pd.read_csv(path, delimiter='\t')

# Select relevant columns
sel = ['Year', 'Leading Cancer Sites', 'MSA', 'Count', 'Population', 'Age-Adjusted Rate']
cancer = cancer[sel]
cancer.columns = [c.replace(' ', '_').lower() for c in cancer.columns]

# Pivot criteria
d = pd.get_dummies(cancer['leading_cancer_sites'], prefix='cancer')
d = d.multiply(cancer['count'], axis="index")
cancer = pd.concat([cancer, d], axis=1).drop(['leading_cancer_sites', 'count'], axis=1)

# Fix MSA
tmp = cancer.msa
ntmp = []

# Also load manual map
manual = {}
with open('data/cancer_manual_map.txt', 'r') as fp:
    for l in fp:
        items = l.split('|')
        manual[items[0].strip()] = items[1].strip()

# MSA Conversion Policy
for t in tmp:
    t = str(t)
    t = t.replace('-', ' ')
    if len(re.split(',|,,', t)) == 1:
        ntmp.append(t)
        continue

    states = re.split(',|,,', t)[1]
    state = states.split()[0]

    areas = t.split()
    if len(areas) > 1:
        area = t.split()[0] + ' ' + t.split()[1]
    else:
            area = areas[0]

    tt = area + ", " + state

    if tt in manual:
        ntmp.append(manual[tt])
    else:
        ntmp.append(tt)

cancer.msa = ntmp
cancer = cancer.dropna()

    # Convert to Lat/Lon
loc = list(cancer.msa)

lat = [_get_lat_lon(i)[0] for i in loc]
lon = [_get_lat_lon(i)[1] for i in loc]

cancer['lat'] = _change_precision(lat, prec=0)
cancer['lon'] = _change_precision(lon, prec=0)

    # Fix Schema
cancer = cancer.dropna()
cancer.year = cancer.year.astype(int)
cancer.lat = cancer.lat.astype(int)
cancer.lon = cancer.lon.astype(int)

    # Group up
cancer = cancer.dropna().groupby(by=['year', 'lat', 'lon'], as_index=False).sum()

cancer = cancer.dropna()


### 4. LOAD LIFE EXPECTANCY DATA

In order to get the life expectancy dataset into a common format, we will be performing following transformations:

1. Clean the column names to remove spaces and capitals
2. Select relevant columns 
3. Fix the Life Exp range to Min and Max values
4. Group by state and county, aggregate by mean to get the average life exp for that state and county
5. Convert state and county to lat and lon using the JSON file
6. Fix the schema for the columns which will become the index (lat, lon, year)
7. Change precision of lat and lon
8. Regroup by year, lat, lon and aggregate by sum

In [ ]:
path = os.path.join('data/raw/', 'health', 'U.S._Life_Expectancy_at_Birth_by_State_and_Census_Tract_-_2010-2015.csv')
life = pd.read_csv(path)

    # Select Relevant
sel = ['State', 'County', 'Life Expectancy',
           'Life Expectancy Range']
life = life[sel].dropna()

    # Fix column names
life.columns = [c.replace(' ', '_').lower() for c in life.columns]

    # Fix Life Exp Range to Min and Max
rng = list(life.life_expectancy_range)
rng_max = [float(str(r).split('-')[1].strip()) for r in rng]
rng_min = [float(str(r).split('-')[0].strip()) for r in rng]

life['life_expectancy_max'] = rng_max
life['life_expectancy_min'] = rng_min
life = life.rename(columns={'life_expectancy': 'life_expectancy_avg'})
life = life.drop('life_expectancy_range', axis=1).dropna()

life = life.groupby(['state', 'county'], as_index=False).mean()

    # Convert to Lat/Lon
print("Getting lat lon list")
loc = list(life.state + ", " + life.county)

lat = [_get_lat_lon(i)[0] for i in loc]
lon = [_get_lat_lon(i)[1] for i in loc]

life['lat'] = _change_precision(lat, prec=0)
life['lon'] = _change_precision(lon, prec=0)

    # Fix Schema
life = life.dropna()
life.lat = life.lat.astype(int)
life.lon = life.lon.astype(int)

    # Group up
life = life.dropna().groupby(by=['lat', 'lon'], as_index=False).sum()

life = life.dropna()


### 5. MERGE ALL DATA

We will now merge the datasets by year, lat, and lon. This is then written to a file that will become our main dataset.

In [ ]:
print("[ LOAD ] Loading AQI...")
print(aqi)

print("[ LOAD ] Loading TRI...")
print(tri)

print("[ LOAD ] Loading Life...")
print(life)

print("[ LOAD ] Loading Cancer...")
print(cancer)

# Join
j = aqi.merge(tri, on=['year', 'lat', 'lon'], how='inner')
j = j.merge(cancer, on=['year', 'lat', 'lon'], how='inner')
j = j.merge(life, on=['lat', 'lon'], how='left')
j = j.dropna()

# TODO: Fix weird behavior with duplicated rows in merge that make this line necessary
j = j.groupby(by=['year', 'lat', 'lon'], as_index=False).mean()

j.to_csv('data/mergeddata.csv', index=False)